In [1]:
%load_ext autoreload  
%autoreload 2 

In [9]:
import ffcv
from ffcv.writer import DatasetWriter
from ffcv.fields import IntField, NDArrayField, FloatField
import datasets
from subset_active_learning.subset_selection import select, preprocess
import wandb
import numpy as np
import torch

In [3]:
processed_ds = preprocess.preprocess_sst2("google/electra-small-discriminator")

No config specified, defaulting to: sst/default
Found cached dataset sst (/home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 760.39it/s]
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-f8fd6701869c12c8.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-92e4d8b08735d65d.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-5fc3a07bc35230ee.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-631cfbbaad84f8a4.arrow
Loading 

In [7]:
LOCAL_PATH = "/home/glai/dev/subset-active-learning/local_bucket"
write_path = f'{LOCAL_PATH}/ds.beton'

# Pass a type for each data field
writer = DatasetWriter(write_path, {
    # Tune options to optimize dataset size, throughput at train-time
    "scalar_label": FloatField(), 
    'input_ids': NDArrayField(np.dtype(np.int32), shape=(8544, 66)), 
    "token_type_ids": NDArrayField(np.dtype(np.int8), shape=(8544, 66)),
    "attention_mask": NDArrayField(np.dtype(np.int8), shape=(8544, 66)),
    'label': IntField()
})

In [10]:
# Convert HF dataset into Torch dataset for ffcv support
class SST2(torch.utils.data.Dataset):
    def __init__(self, hf_ds):
        self.ds = hf_ds
    
    def __getitem__(self, i):
        return self.ds[i]
    
    def __len__(self):
        return len(self.ds)

In [11]:
train_ds = SST2(hf_ds=processed_ds["train"])
# Write dataset
writer.from_indexed_dataset(train_ds)

In [10]:
import time


class BatchSizeComparisonRun: 
    def __init__(self, train_ds: datasets.Dataset, valid_ds: datasets.Dataset, test_ds: datasets.Dataset):
        self.train_ds,self.valid_ds,self.test_ds = train_ds,valid_ds,test_ds

    def _one_run(self, wandb_tag: str, config: select.SubsetTrainingArguments):
        wandb_run = wandb.init(project="subset-search-gpu-opt", entity="johnny-gary", tags=[wandb_tag])
        wandb.log({"batch_size": config.batch_size})
        subset_trainer = select.SubsetTrainer(
            params=config, valid_ds=self.valid_ds, test_ds=self.test_ds
        )
        start_time = time.time()
        subset_trainer.train_one_step(subset=self.train_ds, calculate_test_accuracy=True)
        wandb.log({"run_time": round(time.time() - start_time, 2)})
        wandb_run.finish()

    def run_comparison(self, small_batch_config: select.SubsetTrainingArguments, large_batch_config: select.SubsetTrainingArguments): 
        """
        - train small batch size until early stopping
        - train large batch size until early stopping
        """
        self._one_run(wandb_tag=f"small_batch_{small_batch_config.batch_size}", config=small_batch_config)
        self._one_run(wandb_tag=f"large_batch_{large_batch_config.batch_size}", config=large_batch_config)

In [12]:
INCREASE_FACTOR = 4
small_batch_config = select.SubsetTrainingArguments(batch_size=8, learning_rate=1e-5)
large_batch_config = select.SubsetTrainingArguments(batch_size=small_batch_config.batch_size*INCREASE_FACTOR, learning_rate=small_batch_config.learning_rate*(3/4))

In [13]:
for seed in range(42, 47):
    train_ds = processed_ds["train"].shuffle(seed=seed).select(range(100))
    batch_size_comparison = BatchSizeComparisonRun(train_ds=train_ds, valid_ds=processed_ds["validation"], test_ds=processed_ds["test"])
    batch_size_comparison.run_comparison(small_batch_config=small_batch_config, large_batch_config=large_batch_config)

Loading cached shuffled indices for dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-a7e7c77d48022288.arrow


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,██████▇▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_time,▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▆███████
batch_size,8
loss,0.0014
run_time,237.8
sst2_final_test:accuracy,0.71765
sst2_final_valid:accuracy,0.71117


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,██████▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_time,▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▇██▇
batch_size,32
loss,0.00065
run_time,143.43
sst2_final_test:accuracy,0.70226
sst2_final_valid:accuracy,0.71299


...

Loading cached shuffled indices for dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6b25b3b1c1522c32.arrow


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,████████▇▇▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_time,▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▆▇▆█▆
batch_size,8
loss,0.00419
run_time,160.23
sst2_final_test:accuracy,0.66018
sst2_final_valid:accuracy,0.65668


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,█████▇▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_time,▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▃█▅▅
batch_size,32
loss,0.00063
run_time,143.23
sst2_final_test:accuracy,0.65611
sst2_final_valid:accuracy,0.66848
